In [ ]:
from dataclasses import dataclass
from pathlib import Path

from afft.utils.log import logger
from afft.utils.result import Result

from afft import io

In [ ]:
@dataclass
class SiteVisitData:
    """Class representing site data."""

    label: str
    site: str
    metadata: dict
    entities: list[str]
    components: dict

    def to_dict(self) -> dict:
        """Converts a visit data object to a dictionary."""
        return {
            "label": self.label,
            "site": self.site,
            "metadata": self.metadata,
            "entities": self.entities,
            "components": self.components,
        }


@dataclass
class RevisitSite:
    """Class representing a site index."""

    label: str
    visits: list[SiteVisitData]

    def to_dict(self) -> dict:
        """Converts a site index to a dictionary."""
        data: dict = {
            "site": self.label,
            "visits": {visit.label: visit.to_dict() for visit in self.visits},
        }
        return data


def update_site_visit_data(
    site_name: str, metadata: dict, entities: list, components: dict
) -> SiteVisitData:
    """Creates a visit data object."""

    datetime: str = metadata["datetime"]
    label: str = f"{site_name}_{datetime}"

    site_visit_data: SiteVisitData = SiteVisitData(
        label=label,
        site=site_name,
        metadata=metadata,
        entities=entities,
        components=components,
    )

    return site_visit_data


def update_revisit_site(index: dict) -> RevisitSite:
    """Updates a site index from deprecated index data."""

    updated_visits: list[SiteVisitData] = list()

    for visit_label, visit_data in index["data"].items():

        visit_data: SiteVisitData = update_site_visit_data(
            site_name=index["site"],
            metadata=visit_data["metadata"],
            entities=visit_data["entities"],
            components=visit_data["components"],
        )
        updated_visits.append(visit_data)

    return RevisitSite(index["site"], visits=updated_visits)


def update_site_file(input_file: Path, output_file: Path):
    """Updates a site file."""
    site_index: dict = io.read_json(input_file).unwrap()

    # Create a new site index with updated attributes
    updated_site: RevisitSite = update_revisit_site(site_index)

    write_result: Result[Path, str] = io.write_json(updated_site.to_dict(), output_file)
    if write_result.is_err():
        logger.error(f"failed to write file: {output_path}")
    elif write_result.is_ok():
        logger.info(f"wrote site data to: {write_result.ok()}")

### Configure paths and load index data

In [ ]:
device: Path = Path("/media/martin/pcie_01")
input_directory: Path = Path("acfr_revisits_processed/acfr_index_files")
output_directory: Path = Path("acfr_revisits_processed/acfr_updated_index_files")

filenames: list[str] = [
    "qd61g27j_index.json",
    "qdc5ghs3_index.json",
    "qdch0ftq_index.json",
    "qdchdmy1_index.json",
    "qtqxshxs_index.json",
    "r7jjskxq_index.json",
    "r7jjss8n_index.json",
    "r7jjssbh_index.json",
    "r23m7ms0_index.json",
    "r29mrd5h_index.json",
    "r29mrd12_index.json",
    "r234xgje_index.json",
    "r23685bc_index.json",
]

for filename in filenames:
    input_path: Path = device / input_directory / filename
    output_path: Path = device / output_directory / filename

    update_site_file(input_path, output_path)